In [1]:
# Necessary imports

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.optimize import curve_fit
import scipy.optimize as opt
from tqdm import tqdm
from scipy import signal
from scipy.constants import c
from scipy.ndimage import gaussian_filter
from scipy import ndimage, misc
from astropy.modeling import models, fitting
from astropy.wcs import WCS
import time


import warnings

import numpy as np

import matplotlib.pyplot as plt

from astropy.modeling import models

from astropy import units as u

from specutils.spectra import Spectrum1D, SpectralRegion

from specutils.fitting import fit_generic_continuum

## Subtract out continuum using Median Filter

## Subtract from each wavelength layer a NB image centered at the current layer built by selecting a width 

## NB image is built by computing the median of all layers within chosen width

In [2]:

def subtract_continuum(number,WIDTH,directory,boxes=[],MASK=[]):
    # Subtract out Continuum with Median Filter
    
    # directory: location of fits files to subtract
    # Width of median filter
    
    # boxes: if the user chooses to mask out a region from continuum subtraction, boxes=[x1:x2,y1:y2]

    if(directory == 'psfsub'):
        hdul = fits.open('psfsub/kb220131_'+number+'_tr_psfsub_icubes.fits')
        hdulvar = fits.open('psfsub/kb220131_'+number+'_tr_psfsub_vcubes.fits')
    elif(directory == 'trimcube'):
        hdul = fits.open('trimcube/kb220131_'+number+'_tr_icubes.fits')
        hdulvar = fits.open('trimcube/kb220131_'+number+'_tr_vcubes.fits')
    DATA = hdul[0].data
    header = hdul[0].header
    spacing = header['CD3_3']
    w, ra, dec = np.shape(hdul[0].data)
    first_lamb = header['CRVAL3']
    spacing = header['CD3_3']
    Lambda = np.arange(first_lamb, first_lamb + spacing*w-0.5, spacing)
    hdul.close()
    VAR = hdulvar[0].data
    var_header = hdulvar[0].header
    hdul.close()

    OUT = np.copy(DATA)    
    
    FILTER = ndimage.median_filter(DATA,size=(WIDTH,1,1))
    FILTER[FILTER<0] = 0 # Avoid increasing the flux at a given pixel (no spurious signal)
    
    OUT -= FILTER # subtract
        
    hdu = fits.PrimaryHDU(data=OUT)
    hdu.header = header
    hdu.writeto('psfcontsub/kb220131_'+number+'_tr_psfcontsub_icubes.fits',overwrite=True)
    print('Saved fits file: psfcontsub/kb220131_'+number+'_tr_psfcontsub_icubes.fits')
    
    hdu = fits.PrimaryHDU(data=VAR)
    hdu.header = var_header
    hdu.writeto('psfcontsub/kb220131_'+number+'_tr_psfcontsub_vcubes.fits',overwrite=True)
    

In [3]:
# South
# Pull from psfsub
numbers = [53,54,55,56,57,58,65]
for n in numbers:
    number = str(n)
    subtract_continuum(number,200,'psfsub')

Saved fits file: psfcontsub/kb220131_53_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_54_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_55_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_56_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_57_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_58_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_65_tr_psfcontsub_icubes.fits


In [4]:
# North 
# Pull from trimcube if you did not run psfsub (do not need to since there are no QSOs)
numbers = [59,61,62,63,64,66]
for n in numbers:
    number = str(n)
    subtract_continuum(number,200,'trimcube')

Saved fits file: psfcontsub/kb220131_59_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_61_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_62_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_63_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_64_tr_psfcontsub_icubes.fits
Saved fits file: psfcontsub/kb220131_66_tr_psfcontsub_icubes.fits
